# Week 1 - Measuring Meaning & Sampling
This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze so that we can begin to build our sample. This notebook outlines scraping text from the web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Next, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis. Finally, we ask you to begin building a corpus for preliminary analysis and articulate what your sample represents in context of your final project. 

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. For your final projects, you may find it useful to [read the lucem_illud source code](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/tree/main/lucem_illud) and modify your code for your own interests.

In [ ]:
# %pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package
#lucem_illud is a Latin phrase meaning "that light", the insight we can discover in text data!
#If you get an error like "Access is denied", try running the `pip` command on the command line as an administrator.

If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells (e.g., run `!ls` to display files in the current folder).

There is also a special download required by the `lucem_illud` module in the module `spacy`. You will see this 'en' module later, but you should probably run the following 2 lines of code so you can import `lucem_illud`.

In [12]:
import spacy
# !python -m spacy download en

For this notebook we will be using the following packages:

In [13]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [14]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [15]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [16]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Content analysis - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-cli

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [17]:
dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [18]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom arti


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [19]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [20]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is the study of documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is their non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to au

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [21]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   The data collection instrument used in content...
9   According to current standards of good scienti...
10  Furthermore, the Database of Variables for Con...
11  With the rise of common computing facilities l...
12  By having contents of communication available ...
13  Computer-assisted analysis can help with large...
14  Robert Weber notes: "To make valid inferences ...
15  According to today's reporting standards, quan...
16  There are five types of texts in content analy...
17  Content analysis is rese

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [22]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [23]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [24]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <font color="blue">Exercise 1</font>
<font color="blue">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

For my research question, I'm interested in determining how discussions surrounding reproductive health care and abortion policy changed in news media and policy from before to after the 2022 _Dobbs vs. Jackson_ decision, nationally and instates with the most lenient and most strict abortion policies. The first step of this process is to gain an understanding of abortion access by state across the United States. To do so, I will be examining the Wikipedia page ["Abortion law in the United States by state."](https://en.wikipedia.org/wiki/Abortion_law_in_the_United_States_by_state)

In [25]:
wikipedia_abortion_laws = 'https://en.wikipedia.org/wiki/Abortion_law_in_the_United_States_by_state'
abortion_laws_save = 'wikipedia_abortion_laws.html'

In [26]:
wikiAbortionLawsRequest = requests.get(wikipedia_abortion_laws)
print(wikiAbortionLawsRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Abortion law in the United States by state - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector

In [27]:
wikiAbortionLawsSoup = bs4.BeautifulSoup(wikiAbortionLawsRequest.text, 'html.parser')
print(wikiAbortionLawsSoup.text[:200])





Abortion law in the United States by state - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContent


In [28]:
## find all the h3 tags - these are the US states and territories
abortionH3Tags = wikiAbortionLawsSoup.body.findAll('h3')
for h3Tag in abortionH3Tags[:3]:
    print(h3Tag.text)

Alabama[edit]
Alaska[edit]
American Samoa[edit]


In [34]:
# find all the paragraph content - this is where the state abortion policies will be
abortionPTags = wikiAbortionLawsSoup.body.findAll('p')
for pTag in abortionPTags[:3]:
    print(pTag.text)



The legality of abortion in the United States and the various restrictions imposed on the procedure vary significantly depending on the laws of each state or other jurisdiction. Some states prohibit abortion at all stages of pregnancy with few exceptions, others permit it up to a certain point in a woman's pregnancy, while others allow abortion throughout a woman's pregnancy. In states where abortion is legal, several classes of restrictions on the procedure may exist, such as parental consent or notification laws, requirements that patients be shown an ultrasound before obtaining an abortion, mandatory waiting periods, and counselling requirements.

From 1973 to 2022, Supreme Court rulings in Roe v. Wade (1973) and Planned Parenthood v. Casey (1992), respectively, created and maintained federal protections for a pregnant woman's right to get an abortion, ensuring that states could not ban abortion prior to the point at which a fetus may be deemed viable. However, Roe and Casey were 

In [36]:
## remove the [edit] from the end of each state name
states = []
for h3Tag in abortionH3Tags:
    states.append(re.sub(r'\[edit\]', '', h3Tag.text))
states

['Alabama',
 'Alaska',
 'American Samoa',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'District of Columbia',
 'Delaware',
 'Florida',
 'Georgia',
 'Guam',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Northern Mariana Islands',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'United States Virgin Islands',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 'Limits on abortion',
 'Protections of abortion']

In [37]:
## remove the last two entries since they are not states
states = states[:-2]
states

['Alabama',
 'Alaska',
 'American Samoa',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'District of Columbia',
 'Delaware',
 'Florida',
 'Georgia',
 'Guam',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Northern Mariana Islands',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'United States Virgin Islands',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [59]:
# for each state, find its corresponding policy regarding abortion
abortionStatePolicy = {'state': states, 'policy': [], 'additional details': []}

for state in states:
    state_id = state.replace(' ', '_')

    # find the h3 tag that corresponds to the state
    stateH3Tag = wikiAbortionLawsSoup.body.find('span', id=state_id)
    additionalDetails = ""

    # state tags exists in a span of an h3 element; 
    # need to go up to parent h3 tag to get siblings
    parentTag = stateH3Tag.parent

    # find the next pTag after the h3 element - this is the overall abortion policy for the state/territory
    nextPTag = parentTag.find_next_sibling('p')
    policy = re.sub(r'\[\d+\]', '', nextPTag.get_text()).replace('\n', '')

    # append any additional details
    for sibling in nextPTag.next_siblings:
        if sibling.name == 'h3':
            break
        if sibling.name == 'p':
            additionalDetails += re.sub(r'\[\d+\]', '', sibling.get_text())

    # add policy to dictionary
    abortionStatePolicy['policy'].append(policy)
    if additionalDetails == "":
        additionalDetails = "None"
    abortionStatePolicy['additional details'].append(additionalDetails)
    

In [61]:
# convert to dataframe
abortionStatePolicyDF = pandas.DataFrame.from_dict(abortionStatePolicy)
abortionStatePolicyDF.head()

,state,policy,additional details
0,Alabama,"Abortion is illegal in Alabama, with exception...",Performing an abortion is a Class A felony wit...
1,Alaska,Abortion is legal in Alaska at all stages of p...,Only licensed physicians may perform abortion ...
2,American Samoa,"Abortion is illegal in American Samoa, and was...",None
3,Arizona,Abortion is legal in Arizona up to 15 weeks of...,Patients must meet with a physician at least 2...
4,Arkansas,"Abortion is illegal in Arkansas, with an excep...",Doctors determined to have performed an aborti...



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [62]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [64]:
print(contentPTags[:10])

[<p><b>Content analysis</b> is the study of <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is their non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
</p>, <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a href="/wiki/Semant

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [69]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF.head()

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [66]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [71]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = pandas.concat([contentParagraphsDF, getTextFromWikiPage(*urlTuple)],ignore_index=True)
contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
...,...,...,...,...,...
58,Much of qualitative coding can be attributed t...,https://en.wikipedia.org/wiki/Coding_(social_s...,8,1,assigned labels (sometimes called codes)
59,Coding is considered a process of discovery an...,https://en.wikipedia.org/wiki/Coding_(social_s...,9,1,assigned labels (sometimes called codes)
60,"The process can be done manually, which can be...",https://en.wikipedia.org/wiki/Coding_(social_s...,10,1,assigned labels (sometimes called codes)
61,After assembling codes it is time to organize ...,https://en.wikipedia.org/wiki/Coding_(social_s...,11,1,assigned labels (sometimes called codes)



# <font color="blue">Exercise 2</font>
<font color="blue">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



In [75]:
abortionStatePolicyDF['more information'] = [None] * len(abortionStatePolicyDF['state'])

In [76]:
# for each state, find the corresponding wikipedia page about abortion in that state
for state in states:
    # get the id tag for the state's section
    state_id = state.replace(' ', '_')
    stateH3Tag = wikiAbortionLawsSoup.body.find('span', id=state_id)

    # get the state's parent h3 tag
    parentTag = stateH3Tag.parent

    # find the next div element - this is where the Main article lins is
    nextDivTag = parentTag.find_next_sibling('div')

    # within the div, find the first link - this is the link to the state's wikipedia page
    relurl = nextDivTag.find('a', href=re.compile('/wiki/'), class_=False).get('href')
    state_url = urllib.parse.urljoin(wikipedia_base_url, relurl)
    # add the url to the dataframe
    abortionStatePolicyDF.loc[
        abortionStatePolicyDF['state'] == state, 'more information'] = state_url

In [77]:
abortionStatePolicyDF.head()

,state,policy,additional details,more information
0,Alabama,"Abortion is illegal in Alabama, with exception...",Performing an abortion is a Class A felony wit...,https://en.wikipedia.org/wiki/Abortion_in_Alabama
1,Alaska,Abortion is legal in Alaska at all stages of p...,Only licensed physicians may perform abortion ...,https://en.wikipedia.org/wiki/Abortion_in_Alaska
2,American Samoa,"Abortion is illegal in American Samoa, and was...",None,https://en.wikipedia.org/wiki/Abortion_in_Amer...
3,Arizona,Abortion is legal in Arizona up to 15 weeks of...,Patients must meet with a physician at least 2...,https://en.wikipedia.org/wiki/Abortion_in_Arizona
4,Arkansas,"Abortion is illegal in Arkansas, with an excep...",Doctors determined to have performed an aborti...,https://en.wikipedia.org/wiki/Abortion_in_Arka...


In [80]:
# Navigate to the page for each state and find the first mention of Roe v Wade or Dobbs v Jackson
abortionStatePolicyDF['scotus responses'] = [None] * len(abortionStatePolicyDF['state'])

for index, row in abortionStatePolicyDF.iterrows():
    state = row['state']
    state_url = row['more information']

    # get the page
    stateRequest = requests.get(state_url)
    stateSoup = bs4.BeautifulSoup(stateRequest.text, 'html.parser')
    statePTags = stateSoup.body.findAll('p')

    # find the first mention of Roe v. Wade or Dobbs v. Jackson
    for pTag in statePTags:
        if re.search(r'Roe v. Wade|Dobbs v. Jackson', pTag.text):
            abortionStatePolicyDF.loc[index, 'scotus responses'] = pTag.text
            break

abortionStatePolicyDF.head()



,state,policy,additional details,more information,scotus responses
0,Alabama,"Abortion is illegal in Alabama, with exception...",Performing an abortion is a Class A felony wit...,https://en.wikipedia.org/wiki/Abortion_in_Alabama,"Since 2014, multiple attempts were made to cri..."
1,Alaska,Abortion is legal in Alaska at all stages of p...,Only licensed physicians may perform abortion ...,https://en.wikipedia.org/wiki/Abortion_in_Alaska,Abortion in Alaska is legal at all stages of p...
2,American Samoa,"Abortion is illegal in American Samoa, and was...",None,https://en.wikipedia.org/wiki/Abortion_in_Amer...,None
3,Arizona,Abortion is legal in Arizona up to 15 weeks of...,Patients must meet with a physician at least 2...,https://en.wikipedia.org/wiki/Abortion_in_Arizona,"As a territory, Arizona banned abortion in 186..."
4,Arkansas,"Abortion is illegal in Arkansas, with an excep...",Doctors determined to have performed an aborti...,https://en.wikipedia.org/wiki/Abortion_in_Arka...,The US Supreme Court's decision in 1973's Roe ...


In [91]:
# exemplify a recursive spider by iterating through the links regarding Alabama
# going to the first link on each page
start_url = 'https://en.wikipedia.org/wiki/Abortion_in_Alabama'
topics = []
max_depth = 6

def spiderSearch(url, current_depth=0):
    if current_depth == max_depth:
        return
    else:
        # get the page
        print(f"Current url: {url}")
        request = requests.get(url)
        soup = bs4.BeautifulSoup(request.text, 'html.parser')
        pTags = soup.body.findAll('p')

        topics.append(soup.title.text)

        # find the first wiki link
        for pTag in pTags:
            if pTag.find('a', href=re.compile('/wiki/'), class_=False):
                firstLink = pTag.find('a', href=re.compile('/wiki/'), class_=False).get('href')
                break

        firstLink = urllib.parse.urljoin(wikipedia_base_url, firstLink)

        # recursively call spiderSearch on the first link
        spiderSearch(firstLink, current_depth+1)


In [92]:
spiderSearch(start_url)
print(topics)

Current url: https://en.wikipedia.org/wiki/Abortion_in_Alabama
Current url: https://en.wikipedia.org/wiki/U.S._state
Current url: https://en.wikipedia.org/wiki/United_States
Current url: https://en.wikipedia.org/wiki/North_America
Current url: https://en.wikipedia.org/wiki/Continent
Current url: https://en.wikipedia.org/wiki/Geography
['Abortion in Alabama - Wikipedia', 'U.S. state - Wikipedia', 'United States - Wikipedia', 'North America - Wikipedia', 'Continent - Wikipedia', 'Geography - Wikipedia']


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [93]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[],"uuid":"t:nXFqyQsaizVnIxVAm-ttlA"},"posts-start":0,"posts-total":3926,"posts-type":false,"posts":[{"id":"679413944568430592","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/679413944568430592","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/679413944568430592\/cat-cats-kitty-gato-saturday-meow-katze","type":"photo","date-gmt":"2022-03-22 09:00:29 GMT","date":"Tue, 22 Mar 2022 10:00:29","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1647939629,"format":"html","reblog-key":"MDkw6o5C","slug":"cat-cats-kitty-gato-saturday-meow-katze","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"679413944568430592\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_679413944568430592\"><iframe id=\"like_iframe_679413944568430592\" src=\"https:\/\/assets.tumblr.com\/

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [94]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [95]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [97]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50).head()

,id,photo-url,date,tags,photo-type
0,679413944568430592,https://64.media.tumblr.com/1fb7e346f39e428540...,"Tue, 22 Mar 2022 10:00:29","[gif, lolcat, lolcats, cat, funny, cats, kitty...",gif
1,662815854023655425,https://64.media.tumblr.com/021eac8fbcafbb00a5...,"Mon, 20 Sep 2021 06:00:56","[gif, lolcat, lolcats, cat, funny]",gif
2,662778109891952640,https://64.media.tumblr.com/8c0517adb8c71e4a3d...,"Sun, 19 Sep 2021 20:01:00","[cat, cats, lol, lolcat, lolcats]",png
3,662657302700146688,https://64.media.tumblr.com/061d27cda309d5c809...,"Sat, 18 Sep 2021 12:00:50","[cat, cats, lol, lolcat, lolcats]",jpg
4,662513901538246656,https://64.media.tumblr.com/80584a9d1ff4ddc4fc...,"Thu, 16 Sep 2021 22:01:32","[cat, cats, lol, lolcat, lolcats]",png


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [98]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [99]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [114]:
!pwd

/Users/shaymilner/Library/Mobile Documents/com~apple~CloudDocs/Harris/Winter24/Content Analysis/assignments/soci40133-homeworks/week 1


In [120]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [123]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [124]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
midsummer_nights_dream.txt,A MIDSUMMER NIGHT'S DREAM\n\nby William Shakes...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [125]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂�!���Wn)Ϗ������a�*?p6�Ƒ2?��N����wӍq�A�ڽ|h��z�;%���pƍg�๑���s�����9��fQ�������$����pw��Շ���W[���f��{�M��M�(�M=gã�����l�?��r��s|Ǐ}׏B��o�:E}����N>b�t����{:�M2��I�����
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;4:���ve�WƟ�0�.�y���V�j������bh��i�)�uW������[�jGnG�T,?9�u]��-1�s��7e��.��XU%�ޗ5��[��²�؅���4w�W��N��淼)��R�MT,�,F4��o>n
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@RQ`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [126]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [127]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [128]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [129]:
example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [130]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [131]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [132]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.

# <font color="blue">Exercise 3</font>
<font color="blue">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


In [134]:
# examining the dobs v jackson case
dobs_v_jackson = 'https://www.supremecourt.gov/opinions/21pdf/19-1392_6j37.pdf'

# extract pdf text
dobs_v_jackson_text = readPDF(io.BytesIO(requests.get(dobs_v_jackson, stream=True).content))
print(dobs_v_jackson_text[:1000])

 
(Slip Opinion) 

OCTOBER  TERM,  2021 
 

 

 
Syllabus 

1 

 

NOTE:  Where it is feasible, a syllabus (headnote) will be released, as is 
 
 
being done  in connection with this case, at the time the opinion is issued. 
 
 
The  syllabus  constitutes  no  part  of  the  opinion  of  the  Court  but  has  been 
prepared  by  the  Reporter  of  Decisions  for  the  convenience  of  the  reader. 
See United States v. Detroit Timber & Lumber Co., 200 U. S. 321, 337. 

 
 

 

 

 

 

 

 

 

 

SUPREME COURT OF THE UNITED STATES 

Syllabus 
 

DOBBS, STATE HEALTH OFFICER OF THE 

MISSISSIPPI DEPARTMENT OF HEALTH, ET AL. v. 

JACKSON WOMEN’S HEALTH ORGANIZATION ET AL. 
CERTIORARI TO THE UNITED STATES COURT OF APPEALS FOR 

 

 
THE FIFTH CIRCUIT 

No. 19–1392.  Argued December 1, 2021—Decided June 24, 2022 
 

 

 

 

 

 

 

 

 

Mississippi’s Gestational Age Act provides that “[e]xcept in a medical
 
emergency or in the case of a severe fetal abnormality, a person shall
 
not in

In [149]:
# extract into a dataframe all sentences that contain the word "viability"
dobs_v_jackson_sentences = {'sentence number': [], 'sentence text': []}
for idx, sentence in enumerate(dobs_v_jackson_text.split('.')):
    if re.search(r'viability', sentence):
        # clean the text
        sentence = sentence.strip().strip('\n').replace('\n', ' ').replace('  ', ' ')
        dobs_v_jackson_sentences['sentence number'].append(idx+1)
        dobs_v_jackson_sentences['sentence text'].append(sentence)

dobs_v_jackson_sentences_df = pandas.DataFrame(dobs_v_jackson_sentences)
dobs_v_jackson_sentences_df.head()


,sentence number,sentence text
0,204,An even more glaring defi- ciency was Roe’s fa...
1,207,"The arbitrary viability line, which Casey term..."
2,209,JACKSON WOMEN’S HEALTH ORGANIZATION Syllab...
3,356,"Under this scheme, each trimester of pregnancy..."
4,359,Although the Court acknowledged that States ha...


In [152]:
dobs_v_jackson_sentences_df.loc[1, 'sentence text']

'The arbitrary viability line, which Casey termed Roe’s central rule, has not found  much support among philosophers and ethicists who have attempted to justify a right to abortion'

# <font color="blue">Exercise 4</font>

<font color="blue">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.] 

## ***What?*** 
In my proposed project, I will be analyzing changes in conversations surrounding abortion and preoductive healthcare access before and after the _Dobbs v. Jackson_ supreme court decision in June 2022. Specifically, I will analyze SCOTUS decisions, national and state/local policies, and local and national news media coverage of abortion access to determine how the conversation around access has shifted thematically between the time that of the _Roe_ decision in 1973, to the time leading up to the _Dobbs_ decision, to now. My analysis will provide further illimunation into the social game between political rulings (policy) and public opinion and media coverage (news articles), including the positions of those responsible for passing policy and the responses of those affected by the policy.

## ***Why?***
Analyzing both policy and news coverage of abortion in the United States will provide the opportunity to engage in highly structured (policy) and highly unstructured (news media) data, expanding the breadth and generalizability of my analysis. It will help to further showcase the interactions between national policy and local reactions in the United States, while also presenting the opportunity to think critically on  data collection particularly for news media coverage, given the number of news organizations in the country. I suspect that policy decisions will work to frame news media coverage of abortion, so this sample will enable me to further tease out that hunch with the eventual goal of developing either an inferential or predictive model regarding how news outlets will respond to certain languages present in policy. Some limitations regarding my proposed sample include the relatively small sample of SCOTUS decisions regarding abortion, though the inclusion of state policies will help to fill this out, and the huge sample and highly dimensional nature of news coverage covering a more than 50 year span of time. Alternatives to the approach of using a broad range of news media articles would be focusing solely on policy and how conversations regarding abortion and reproducitve healthcare services have changed in the political sphere, and/or to focus on the most popular news media organizations, though this method could lead to bias. I could scale up my sample by including social media data, such as from Twitter, to get a more micro scale understanding of how conversations regarding abortion are happening in everyday life.

### Sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

